# Analog Input with AITask

Examples of analog input acquisition using the AITask class.

In [ ]:
# Check nidaqmx availability
try:
    import nidaqmx
except ImportError:
    raise RuntimeError(
        "nidaqmx is not installed. Install with: pip install nidaqmx"
    )

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from nidaqwrapper import AITask, list_devices

# Show connected hardware
devices = list_devices()
print(f"Found {len(devices)} device(s):\n")
for i, dev in enumerate(devices):
    print(f"Device {i}: {dev['name']} ({dev['product_type']})")

# Edit these to match your hardware
device_ind = 0
channel_ind = 0

## Single Voltage Channel

Acquire from one voltage channel and plot the result.

In [ ]:
task = AITask("voltage_test", sample_rate=1000.0)
task.add_channel(
    channel_name="voltage_0",
    device_ind=device_ind,
    channel_ind=channel_ind,
    units="V"
)
task.start(start_task=True)

# Wait for buffer to fill
time.sleep(0.5)

# acquire_base() returns (n_channels, n_samples)
data = task.acquire_base()
task.clear_task()

print(f"Acquired shape: {data.shape}")
print(f"Channels: {data.shape[0]}, Samples: {data.shape[1]}")

# Plot
plt.figure(figsize=(10, 4))
plt.plot(data[0])  # First (and only) channel
plt.xlabel("Sample")
plt.ylabel("Voltage (V)")
plt.title("Single Voltage Channel")
plt.grid(True)
plt.show()

## Accelerometer (IEPE)

Acquire from an IEPE accelerometer with sensitivity and units.

In [ ]:
task = AITask("accel_test", sample_rate=25600.0)
task.add_channel(
    channel_name="accel_z",
    device_ind=device_ind,
    channel_ind=channel_ind,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)
task.start(start_task=True)

time.sleep(0.5)

data = task.acquire_base()
task.clear_task()

print(f"Acquired shape: {data.shape}")

plt.figure(figsize=(10, 4))
plt.plot(data[0])
plt.xlabel("Sample")
plt.ylabel("Acceleration (g)")
plt.title("IEPE Accelerometer")
plt.grid(True)
plt.show()

## Multi-Channel

Acquire from three channels and plot separately.

In [ ]:
task = AITask("multi_channel", sample_rate=1000.0)

# Add three voltage channels
for i in range(3):
    task.add_channel(
        channel_name=f"ch_{i}",
        device_ind=device_ind,
        channel_ind=i,
        units="V"
    )

task.start(start_task=True)
time.sleep(0.5)

data = task.acquire_base()
task.clear_task()

print(f"Acquired shape: {data.shape}")

# Subplot for each channel
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
for i in range(3):
    axes[i].plot(data[i])
    axes[i].set_ylabel(f"Ch {i} (V)")
    axes[i].grid(True)
axes[-1].set_xlabel("Sample")
plt.suptitle("Multi-Channel Acquisition")
plt.tight_layout()
plt.show()

## Context Manager

Use `with` statement for automatic cleanup.

In [ ]:
with AITask("context_test", sample_rate=1000.0) as task:
    task.add_channel(
        channel_name="voltage_0",
        device_ind=device_ind,
        channel_ind=channel_ind,
        units="V"
    )
    task.start(start_task=True)
    time.sleep(0.5)
    data = task.acquire_base()
    print(f"Acquired {data.shape[1]} samples")

# Task is automatically cleared when exiting the with block

## Custom Scaling

Apply linear scaling to voltage channels: `output = slope * input + intercept`

In [ ]:
# Example 1: Scale with slope only (y_intercept=0)
task = AITask("scale_slope", sample_rate=1000.0)
task.add_channel(
    channel_name="scaled_1",
    device_ind=device_ind,
    channel_ind=channel_ind,
    units="custom_unit",
    scale=10.0  # Multiply voltage by 10
)
task.start(start_task=True)
time.sleep(0.5)
data1 = task.acquire_base()
task.clear_task()

print(f"Scale with slope=10.0: {data1.shape}")

# Example 2: Scale with slope and intercept
task = AITask("scale_both", sample_rate=1000.0)
task.add_channel(
    channel_name="scaled_2",
    device_ind=device_ind,
    channel_ind=channel_ind,
    units="custom_unit",
    scale=(5.0, 2.0)  # output = 5.0 * voltage + 2.0
)
task.start(start_task=True)
time.sleep(0.5)
data2 = task.acquire_base()
task.clear_task()

print(f"Scale with slope=5.0, intercept=2.0: {data2.shape}")

## Save/Load Config

Serialize task configuration to TOML and reload it.

In [ ]:
# Create and configure a task
task = AITask("config_test", sample_rate=1000.0)
task.add_channel(
    channel_name="voltage_0",
    device_ind=device_ind,
    channel_ind=channel_ind,
    units="V",
    min_val=-10.0,
    max_val=10.0
)
task.add_channel(
    channel_name="voltage_1",
    device_ind=device_ind,
    channel_ind=channel_ind + 1,
    units="V"
)

# Save to TOML file (relative path)
config_path = "ai_config.toml"
task.save_config(config_path)
task.clear_task()

print(f"Configuration saved to {config_path}\n")

# Show the file content
with open(config_path, "r") as f:
    content = f.read()
    print("File content:")
    print(content)

In [ ]:
# Load the configuration back
reloaded_task = AITask.from_config(config_path)

print(f"Reloaded task: {reloaded_task.task_name}")
print(f"Sample rate: {reloaded_task.sample_rate} Hz")
print(f"Channels: {reloaded_task.channel_list}")
print(f"Number of channels: {reloaded_task.number_of_ch}")

reloaded_task.clear_task()